In [1]:
import pandas as pd
import numpy as np

import time


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC       

In [2]:

driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.co.kr/maps/place/Goseong+Unification+Observatory+Tower/data=!4m7!3m6!1s0x5fd85137bc21b7cd:0x89f99cd74c63c783!8m2!3d38.5866769!4d128.3752316!9m1!1b1?hl=en")
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Sort\']'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()


In [ ]:
'''
# specific attraction is differ from others, so we change html code for it.
driver = webdriver.Chrome('C:/Users/YOONSU/chromedriver.exe')
driver.get("https://www.google.co.kr/maps/place/Daemyung+Delpino+Golf+%26+Resort+%26+Hotel/data=!4m14!1m2!2m1!1z64247ZS864W4IOqzqO2UhOyVpOumrOyhsO2KuCDriIjsjbDrp6TsnqU!3m10!1s0x5fd8bd426d38d6fb:0x5cd1daca122d04bf!5m2!4m1!1i2!8m2!3d38.2120109!4d128.4940943!9m1!1b1!15sCinrjbjtlLzrhbgg6rOo7ZSE7JWk66as7KGw7Yq4IOuIiOyNsOunpOyepVouIizrjbjtlLzrhbgg6rOo7ZSEIOyVpCDrpqzsobDtirgg64iI7I2w66ekIOyepZIBDHJlc29ydF9ob3RlbJoBI0NoWkRTVWhOTUc5blMwVkpRMEZuU1VOWGNrNTZYMGwzRUFF?hl=en")
driver.maximize_window()
time.sleep(5)

# sort to recent review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9) > button:nth-child(2) > div.IAbLGd'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements(By.XPATH,
                                     '//li[@role=\'menuitemradio\']')[1]
recent_rating_bt.click()
'''

In [3]:
# scroll code
# total n review (10 reviews per 1 page)
from tqdm import tqdm

# google map allows crawling until 93th scroll(maybe robots.txt disallow crawling)
page_num = 93
per = 10
pbar = tqdm(total=page_num*per)
with tqdm(total=page_num*per, position=0, leave=True) as pbar:
    for i in range(page_num):
        buffer = driver.find_elements_by_class_name('qjESne') # buffer location
        action = ActionChains(driver)

        n =len(buffer)
        action.move_to_element(buffer[n-1]).perform() # move to last buffer
        time.sleep(1)
        pbar.update(per)
    pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████| 930/930 [02:21<00:00,  6.59it/s]


In [4]:
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

boxes = driver.find_elements_by_class_name('jftiEf.fontBodyMedium') # review class


# more butten click
for box in boxes:
    try:
        more_btn = box.find_element_by_class_name('w8nwRe.kyuRq')
        more_btn.click()
        time.sleep(0.5)
    except:
        continue
# review collection
for box in boxes:

    IDs = box.find_elements_by_class_name('d4r55')

    Dates = box.find_elements_by_class_name('rsqaWe')

    Stars = box.find_elements_by_class_name('kvMYJc')

    reviews = box.find_elements_by_css_selector('span.wiI7pd')

    for ID, Date, Star, review in zip(IDs, Dates, Stars, reviews):
        name_list.append(ID.text)
        date_list.append(Date.text)
        stars_list.append(Star.get_attribute('aria-label'))
        review_list.append(review.text)


  0%|                                                                                          | 0/930 [02:30<?, ?it/s]


In [15]:
# review list to dataframe
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list
    })
pd.set_option('display.max_rows', None)
print(len(review))
review.head()

890


,name,rating,review,date
0,Ko Taesuck,5 stars,,42 minutes ago
1,이지민짱,5 stars,,2 hours ago
2,황해성,4 stars,,18 hours ago
3,서제민,3 stars,(Translated by Google) It was just nice to be ...,19 hours ago
4,Tom Teacher,3 stars,"Nothing much to see tbh, beside some mountains...",20 hours ago


In [32]:
eng_review = review.copy()

In [33]:
# remove korean name
eng_review['name2'] = eng_review['name'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['name2'].str.len() == 0]

# remove korean review
eng_review['review2'] = eng_review['review'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['review2'].str.len() == 0]

# remove name2 & review2 columns
eng_review.drop(['name2','review2'],axis=1, inplace=True)

# remove unreviewed data
eng_review = eng_review[eng_review['review'].str.len()!=0]


# remove "(Translated by Google)" sentence
eng_review['review'] = eng_review['review'].str.replace('(Translated by Google) ','',regex=False)

# remove "Original" sentence
eng_review['review'] = eng_review['review'].str.replace('\(Original\)','',regex=False)

# remove not english sentence
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# delete side blank and "\n"
eng_review['review'] = eng_review['review'].str.strip()

# remove special characters
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# replace with lowercase
eng_review['review'] = eng_review['review'].str.lower()

print(len(eng_review))
eng_review.head()

27


,name,rating,review,date
4,Tom Teacher,3 stars,nothing much to see tbh beside some mountains...,20 hours ago
22,M V CHINNAMANI,5 stars,very good,3 days ago
97,Bezalel Gani,5 stars,was a great view loved it,2 weeks ago
153,James Kim,5 stars,i took my children to this place on 25th of ju...,a month ago
158,Fabian,3 stars,except for beautiful landscape there is not mu...,a month ago


In [34]:
eng_review.tail(10)

,name,rating,review,date
582,Jennifer Gilbert,5 stars,on a clear day you can see the dmz as well as...,8 months ago
599,SOON CHUL WOO,5 stars,it was a great experience which all koreans w...,9 months ago
643,Atsushi Harada,5 stars,an observatory overlooking north korea from th...,10 months ago
678,Roy Whitworth,4 stars,well worth a visit see the coastline and i t...,a year ago
715,Jeff Sheets,5 stars,very scenic definitely worth visiting and lea...,a year ago
759,Frédéric Weber,5 stars,a must see if you are nearby seokcho beautifu...,a year ago
768,Pat,5 stars,note you have to get a pass to cross into the...,a year ago
785,Zo Zo,5 stars,nice and unique scenery,a year ago
869,Muhammad Umar Farooq,5 stars,a very nice place to have a sneak peek at de m...,a year ago
886,JAE IL Ban,4 stars,interesting place to visit,a year ago


In [35]:
eng_review.to_csv("고성통일전망대.csv", encoding="utf-8")

==================================================================================